## Imports

In [1]:
import datetime
import gc

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *

In [3]:
cuda_use_gpus(1)

In [4]:
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


## Config

In [5]:
feature_list_id = 'oofp_currie32_cnn_magic'

In [6]:
RANDOM_SEED = 42

In [7]:
np.random.seed(RANDOM_SEED)

## Read Data

In [8]:
embedding_matrix = load(aux_data_folder + 'embedding_weights_fasttext_filtered_no_stopwords.pickle')

In [9]:
X_train_q1 = load(features_data_folder + 'X_train_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_train_q2 = load(features_data_folder + 'X_train_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [10]:
X_test_q1 = load(features_data_folder + 'X_test_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_test_q2 = load(features_data_folder + 'X_test_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [11]:
magic_feature_lists = [
    'magic_jturkewitz',
    'magic_stas_svd_150',
    'magic_stas_avito',
]

In [12]:
X_train_magic, X_test_magic, _ = load_feature_lists(magic_feature_lists)

In [13]:
X_train_magic = X_train_magic.values
X_test_magic = X_test_magic.values

In [14]:
scaler = StandardScaler()
scaler.fit(np.vstack([X_train_magic, X_test_magic]))
X_train_magic = scaler.transform(X_train_magic)
X_test_magic = scaler.transform(X_test_magic)

In [15]:
y_train = load(features_data_folder + 'y_train.pickle')

In [16]:
EMBEDDING_DIM = embedding_matrix.shape[-1]
VOCAB_LENGTH = embedding_matrix.shape[0]
MAX_SEQUENCE_LENGTH = X_train_q1.shape[-1]

In [17]:
print(EMBEDDING_DIM, VOCAB_LENGTH, MAX_SEQUENCE_LENGTH)

300 101442 30


## Train Models & Compute Out-of-Fold Predictions

In [18]:
init_weights = initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=2)
init_bias = 'zeros'

In [19]:
def create_embedding_block():
    input_seq = Input(shape=(MAX_SEQUENCE_LENGTH, ), dtype='int32')
    
    embedding_seq = Embedding(
        VOCAB_LENGTH,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    )(input_seq)
    
    output_seq = embedding_seq
    return input_seq, output_seq

In [20]:
def create_model_question_conv_branch(input_seq, params):
    conv_1 = Conv1D(
        params['num_conv_filters'],
        kernel_size=params['conv_kernel_size'],
        padding='same',
    )(input_seq)
    
    bn_1 = BatchNormalization()(conv_1)
    relu_1 = Activation('relu')(bn_1)
    dropout_1 = Dropout(params['conv_dropout_rate'])(relu_1)

    conv_2 = Conv1D(
        params['num_conv_filters'],
        kernel_size=params['conv_kernel_size'],
        padding='same',
    )(dropout_1)
    
    bn_2 = BatchNormalization()(conv_2)
    relu_2 = Activation('relu')(bn_2)
    dropout_2 = Dropout(params['conv_dropout_rate'])(relu_2)
    
    flatten = Flatten()(dropout_2)
    output = flatten
    
    return output

In [21]:
def create_model_question_timedist_max_branch(input_seq, params):
    timedist = TimeDistributed(Dense(EMBEDDING_DIM))(input_seq)
    bn = BatchNormalization()(timedist)
    relu = Activation('relu')(bn)
    dropout = Dropout(params['timedist_dropout_rate'])(relu)

    lambda_max = Lambda(
        lambda x: K.max(x, axis=1),
        output_shape=(EMBEDDING_DIM, )
    )(dropout)
    
    output = lambda_max
    return output

In [22]:
def create_dense_block(input_layer, num_units, dropout_rate):
    dense = Dense(
        num_units,
        kernel_initializer=init_weights,
        bias_initializer=init_bias,
    )(input_layer)
    bn = BatchNormalization()(dense)
    relu = Activation('relu')(bn)
    dropout = Dropout(dropout_rate)(relu)
    output = dropout
    
    return output

In [23]:
def create_model(params):
    input_q1, emb_q1 = create_embedding_block()
    input_q2, emb_q2 = create_embedding_block()
    
    # Feature extractors.
    conv_q1_output = create_model_question_conv_branch(emb_q1, params)
    conv_q2_output = create_model_question_conv_branch(emb_q2, params)
    
    timedist_q1_output = create_model_question_timedist_max_branch(emb_q1, params)
    timedist_q2_output = create_model_question_timedist_max_branch(emb_q2, params)
    
    # Mid-level transforms.
    conv_merged = concatenate([conv_q1_output, conv_q2_output])
    conv_dense_1 = create_dense_block(conv_merged, params['num_dense_1'], params['dense_dropout_rate'])
    conv_dense_2 = create_dense_block(conv_dense_1, params['num_dense_2'], params['dense_dropout_rate'])

    td_merged = concatenate([timedist_q1_output, timedist_q2_output])
    td_dense_1 = create_dense_block(td_merged, params['num_dense_1'], params['dense_dropout_rate'])
    td_dense_2 = create_dense_block(td_dense_1, params['num_dense_2'], params['dense_dropout_rate'])

    # Magic features.
    magic_input = Input(shape=(X_train_magic.shape[-1], ))
    
    # Main dense block.
    merged_main = concatenate([conv_dense_2, td_dense_2, magic_input])
    dense_main_1 = create_dense_block(merged_main, params['num_dense_1'], params['dense_dropout_rate'])
    dense_main_2 = create_dense_block(dense_main_1, params['num_dense_2'], params['dense_dropout_rate'])
    dense_main_3 = create_dense_block(dense_main_2, params['num_dense_3'], params['dense_dropout_rate'])
    
    output = Dense(
        1,
        kernel_initializer=init_weights,
        bias_initializer=init_bias,
        activation='sigmoid',
    )(dense_main_3)
    
    model = Model(
        inputs=[input_q1, input_q2, magic_input],
        outputs=output,
    )
    
    model.compile(
        loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['accuracy']
    )

    return model

In [24]:
model_params = {
    'num_conv_filters': 32,
    'num_dense_1': 256,
    'num_dense_2': 128,
    'num_dense_3': 100,
    'conv_kernel_size': 3,
    'conv_dropout_rate': 0.25,
    'timedist_dropout_rate': 0.25,
    'dense_dropout_rate': 0.25,
}

In [25]:
model_checkpoint_path = aux_data_folder + 'fold-checkpoint-' + feature_list_id + '.h5'

In [26]:
NUM_FOLDS = 5
NUM_EPOCHS = 200

In [27]:
kfold = StratifiedKFold(
    n_splits=NUM_FOLDS,
    shuffle=True,
    random_state=RANDOM_SEED
)

In [28]:
y_train_oofp = np.zeros_like(y_train, dtype='float64')

In [29]:
y_test_oofp = np.zeros((len(X_test_q1), NUM_FOLDS))

In [30]:
def predict(model, X_q1, X_q2, X_magic):
    y1 = model.predict(
        [X_q1, X_q2, X_magic],
        batch_size=1024,
        verbose=1
    ).reshape(-1)
    
    y2 = model.predict(
        [X_q2, X_q1, X_magic],
        batch_size=1024,
        verbose=1
    ).reshape(-1)
    
    return (y1 + y2) / 2

In [31]:
%%time

for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train_q1, y_train)):
    X_fold_train_q1 = np.vstack([X_train_q1[ix_train], X_train_q2[ix_train]])
    X_fold_train_q2 = np.vstack([X_train_q2[ix_train], X_train_q1[ix_train]])
    X_fold_train_magic = np.vstack([X_train_magic[ix_train], X_train_magic[ix_train]])

    X_fold_val_q1 = np.vstack([X_train_q1[ix_val], X_train_q2[ix_val]])
    X_fold_val_q2 = np.vstack([X_train_q2[ix_val], X_train_q1[ix_val]])
    X_fold_val_magic = np.vstack([X_train_magic[ix_val], X_train_magic[ix_val]])

    y_fold_train = np.concatenate([y_train[ix_train], y_train[ix_train]])
    y_fold_val = np.concatenate([y_train[ix_val], y_train[ix_val]])
    
    print()
    print(f'Fitting fold {fold_num + 1} of {kfold.n_splits}')
    print()
    
    model = create_model(model_params)
    model.fit(
        [X_fold_train_q1, X_fold_train_q2, X_fold_train_magic], y_fold_train,
        validation_data=([X_fold_val_q1, X_fold_val_q2, X_fold_val_magic], y_fold_val),

        batch_size=64,
        epochs=NUM_EPOCHS,
        verbose=1,
        
        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.001,
                patience=3,
                verbose=1,
                mode='auto',
            ),
            ModelCheckpoint(
                model_checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                verbose=2,
            ),
        ],
    )
        
    # Create out-of-fold prediction.
    model.load_weights(model_checkpoint_path)
    
    y_train_oofp[ix_val] = predict(model, X_train_q1[ix_val], X_train_q2[ix_val], X_train_magic[ix_val])
    y_test_oofp[:, fold_num] = predict(model, X_test_q1, X_test_q2, X_test_magic)
    
    # Clear GPU memory.
    K.clear_session()
    del X_fold_train_q1
    del X_fold_train_q2
    del X_fold_train_magic
    del X_fold_val_q1
    del X_fold_val_q2
    del X_fold_val_magic
    del model
    gc.collect()


Fitting fold 1 of 5

Train on 646862 samples, validate on 161718 samples
Epoch 1/200
646862/646862 [==============================] - 229s - loss: 0.3511 - acc: 0.8451 - val_loss: 0.3213 - val_acc: 0.8555
Epoch 2/200
646862/646862 [==============================] - 255s - loss: 0.3171 - acc: 0.8572 - val_loss: 0.2985 - val_acc: 0.8664
Epoch 3/200
646862/646862 [==============================] - 260s - loss: 0.3010 - acc: 0.8654 - val_loss: 0.2905 - val_acc: 0.8712
Epoch 4/200
646862/646862 [==============================] - 260s - loss: 0.2890 - acc: 0.8714 - val_loss: 0.2853 - val_acc: 0.8735
Epoch 5/200
646862/646862 [==============================] - 254s - loss: 0.2790 - acc: 0.8761 - val_loss: 0.2847 - val_acc: 0.8762
Epoch 6/200
646862/646862 [==============================] - 261s - loss: 0.2719 - acc: 0.8795 - val_loss: 0.3005 - val_acc: 0.8713
Epoch 7/200
646862/646862 [==============================] - 250s - loss: 0.2653 - acc: 0.8826 - val_loss: 0.2817 - val_acc: 0.8761
Ep

646862/646862 [==============================] - 179s - loss: 0.2525 - acc: 0.8886 - val_loss: 0.2778 - val_acc: 0.8779
Epoch 10/200
646862/646862 [==============================] - 179s - loss: 0.2478 - acc: 0.8910 - val_loss: 0.2951 - val_acc: 0.8728
Epoch 11/200
646862/646862 [==============================] - 179s - loss: 0.2439 - acc: 0.8928 - val_loss: 0.2936 - val_acc: 0.8731
Epoch 12/200
646862/646862 [==============================] - 179s - loss: 0.2399 - acc: 0.8949 - val_loss: 0.2882 - val_acc: 0.8775
Epoch 13/200
646862/646862 [==============================] - 179s - loss: 0.2361 - acc: 0.8965 - val_loss: 0.2776 - val_acc: 0.8791
Epoch 00012: early stopping
2343936/2345796 [============================>.] - ETA: 0s
Fitting fold 3 of 5

Train on 646864 samples, validate on 161716 samples
Epoch 1/200
646864/646864 [==============================] - 179s - loss: 0.3506 - acc: 0.8450 - val_loss: 0.3178 - val_acc: 0.8570
Epoch 2/200
646864/646864 [=============================

646866/646866 [==============================] - 176s - loss: 0.2881 - acc: 0.8717 - val_loss: 0.2835 - val_acc: 0.8727
Epoch 5/200
646866/646866 [==============================] - 176s - loss: 0.2791 - acc: 0.8758 - val_loss: 0.2786 - val_acc: 0.8753
Epoch 6/200
646866/646866 [==============================] - 176s - loss: 0.2718 - acc: 0.8791 - val_loss: 0.2710 - val_acc: 0.8800
Epoch 7/200
646866/646866 [==============================] - 176s - loss: 0.2650 - acc: 0.8826 - val_loss: 0.2746 - val_acc: 0.8781
Epoch 8/200
646866/646866 [==============================] - 175s - loss: 0.2597 - acc: 0.8854 - val_loss: 0.2743 - val_acc: 0.8782
Epoch 9/200
646866/646866 [==============================] - 167s - loss: 0.2539 - acc: 0.8880 - val_loss: 0.2753 - val_acc: 0.8800
Epoch 10/200
646866/646866 [==============================] - 173s - loss: 0.2492 - acc: 0.8903 - val_loss: 0.2767 - val_acc: 0.8811
Epoch 00009: early stopping
2343936/2345796 [============================>.] - ETA: 0s


In [33]:
cv_score = log_loss(y_train, y_train_oofp)
print('CV score:', cv_score)

CV score: 0.266362794559


## Save feature names

In [34]:
feature_names = [
    'oofp_currie32_cnn_magic',
]

In [35]:
save_lines(feature_names, features_data_folder + f'X_train_{feature_list_id}.names')

## Save Train features

In [36]:
y_train_oofp = y_train_oofp.reshape((-1, 1))

In [37]:
save(y_train_oofp, features_data_folder + f'X_train_{feature_list_id}.pickle')

## Save Test features

In [38]:
y_test_oofp_mean = np.mean(y_test_oofp, axis=1).reshape((-1, 1))

In [39]:
save(y_test_oofp_mean, features_data_folder + f'X_test_{feature_list_id}.pickle')